In [23]:
!pip install mysql.connector

In [5]:
#Передаём строковые значения параметров для запроса. В идеале, это должен быть результат обработки запроса GET от страницы
#через который всё преобразуется в такую строку.
#Пример работы на is_enternal: is_enternal='0,1' - это оба, is_enternal='1' - энтеральное, is_enternal='1' - неэнетеральное.
#Так получаме набор тэгов для встраивания в строку в следующей функции
is_enternal='0,1'
polim='3,4'
caloric='5,6,7'
special='21, 10, 11, 12, 13'
vol='8,9'

In [2]:
#Функия для получения таблицы с именами, производителем, энтеральностью и доступностью веществ по заявленым тэгам
def get_by_tags (polim,caloric,special,vol,is_enternal):
    return (f"SELECT n0.id as nutrient_id, n0.name, n0.manufacturer, n0.is_enteral, n0.is_available FROM `nutrition` n0 INNER JOIN `nutrition_tagging` n1 ON n0.id = n1.nutrient_id and n1.tag_id in ({polim}) INNER JOIN `nutrition_tagging` n2 ON n1.nutrient_id = n2.nutrient_id and n2.tag_id in ({caloric}) INNER JOIN `nutrition_tagging` n3 ON n3.nutrient_id = n2.nutrient_id and n3.tag_id in ({special}) INNER JOIN `nutrition_tagging` n4 ON n4.nutrient_id = n3.nutrient_id and n4.tag_id in ({vol}) INNER JOIN `nutrition` n5 ON n0.name = n5.name and n5.is_enteral in ({is_enternal});")

In [3]:
#Функия для получения таблицы со списокм всех тэгов, конкатенированных для каждого нутриента
def get_nutrient_tags ():
    return ('select n1.nutrient_id, GROUP_CONCAT(CONCAT(n2.tag)) as "all tags" from nutrition_tagging n1 INNER JOIN nutrition_tag n2 ON n1.tag_id=n2.id GROUP BY n1.nutrient_id;')

In [4]:
#Функия для получения таблицы пищевой и энергетической ценностей для каждого нутриента
def get_PFCE_mash_positive ():
    return ('select n2.nutrient_id, n1.id as feature_id, n1.name, n2.value from nutrition_feature n1 INNER JOIN nutrition_feature_value n2 on n1.id=n2.feature_id where id in (16, 46, 29, 49, 50, 51, 13, 48);')

In [7]:
##Запрос и запись в датафреймы всех необходимых для финальной таблицы данных из БД
import mysql.connector
from mysql.connector import connect, Error
import pandas as pd
from pandas import DataFrame
DB_NAME = "oncology_nutrition"
DB_USER = "nutrition_list"
DB_PASSWD = "nutrListWhiteForest"
DB_HOST = "projectswhynot.site"
connection = connect(host=DB_HOST,
                user=DB_USER,
                password=DB_PASSWD,
                database=DB_NAME,
                port=11459)
cursor = connection.cursor()
cursor.execute(get_by_tags (polim,caloric,special,vol,is_enternal))

nutr_search_res = DataFrame(cursor.fetchall())
nutr_search_res.columns = cursor.column_names

cursor.execute(get_nutrient_tags())

nutr_tags = DataFrame(cursor.fetchall())
nutr_tags.columns = cursor.column_names

cursor.execute(get_PFCE_mash_positive())

PFCE_values_mash= DataFrame(cursor.fetchall())
PFCE_values_mash.columns = cursor.column_names


In [31]:
##Слияние датавреймов чтобы было нормально и читаемо
P_values=PFCE_values_mash[PFCE_values_mash['name']=='Белок/100мл']
F_values=PFCE_values_mash[PFCE_values_mash['name']=='Жиры/100мл']
C_values=PFCE_values_mash[PFCE_values_mash['name']=='Углеводы/100мл']
E_values=PFCE_values_mash[(PFCE_values_mash['name']=='Ккал/100мл') | (PFCE_values_mash['name']=='ККал/100мл')]
PFCE_values=pd.merge(pd.merge(P_values,F_values,on='nutrient_id'),pd.merge(C_values,E_values,on='nutrient_id'),on='nutrient_id')

In [41]:
##Слияние всего в сводную таблицу
data=pd.merge(pd.merge(nutr_search_res,PFCE_values,on='nutrient_id'),nutr_tags,on='nutrient_id')
data=data.rename(columns={"name": "Название", "manufacturer": "Производитель", "is_enteral": "ЭП/ПЭП", "value_x_x":"Белков в 100 мл", "value_y_x":"Жиров в 100 мл","value_x_y":"Углеводов в 100 мл","value_y_y":"ККал в 100 мл", "all tags": "Характеристики","is_available": "Наличие в клинике"})
data=data.drop(['feature_id_x_x', 'name_x_x','nutrient_id','feature_id_y_x', 'name_y_x', 'feature_id_x_y', 'name_x_y','feature_id_y_y', 'name_y_y'], axis=1)
cols=['Название',
 'Производитель',
 'ЭП/ПЭП',
 'Белков в 100 мл',
 'Жиров в 100 мл',
 'Углеводов в 100 мл',
 'ККал в 100 мл',
 'Характеристики','Наличие в клинике']
data = data[cols]

,Название,Производитель,ЭП/ПЭП,Белков в 100 мл,Жиров в 100 мл,Углеводов в 100 мл,ККал в 100 мл,Характеристики,Наличие в клинике
4,Нутридринк (банан),Нутриция,1,5.9,5.8,18.4,150,"Сипинг,Полимерная,Без пищевых волокон,Без проб...",1


In [ ]:
##безуспешная попытка передать что-то на страницу

from flask import Flask, request, render_template
app = Flask(__name__)
@app.route('/', methods =["GET"])
def index():
    if request.method == "POST":
       # getting input with name = fname in HTML form
       nutr_type = request.form.get("nutr_type")
       # getting input with name = lname in HTML form
       polymer = request.form.get("polymer")
       return nutr_type+polymer
    return render_template("index.html", nutritions=['1','2','3'])
if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2023 02:12:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 02:12:00] "GET /pages/index.css HTTP/1.1" 404 -
127.0.0.1 - - [17/Jul/2023 02:13:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 02:13:00] "GET /pages/index.css HTTP/1.1" 404 -
